In [1]:
# Tests the new function for reshaping dtrajs when supplied a n_samples,2

In [2]:
import numpy as np
from msmbuilder.cluster import NDGrid
from pyemma.msm import MaximumLikelihoodMSM
from pyemma.coordinates.util import DtrajReshape 

/Users/robert_arbon/anaconda/envs/ml4dyn/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/robert_arbon/anaconda/envs/ml4dyn/lib/python3.5/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
/Users/robert_arbon/anaconda/envs/ml4dyn/lib/python3.5/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pa

In [19]:
all_X = [np.load('../data/000.5pc/quad_well_00.npy'), np.load('../data/000.5pc/quad_well_01.npy')]

In [20]:
tau = 25

In [21]:
X_split = []
for X in all_X:
    for i in range(X.shape[0]-tau):
        X_split.append(list(*X[[i,i+tau], :].T))

In [22]:
X_split = np.array(X_split)

In [27]:
X_split.shape

# np.save('split_example.npy', X_split)

(794, 2)

In [28]:
X_split[:5]

array([[ 0.90180361,  0.48055044],
       [ 0.96266208,  0.48757914],
       [ 0.99190124,  0.58029218],
       [ 0.90946687,  0.61787998],
       [ 0.94062025,  0.58357143]])

In [29]:
X_split[-5:]

array([[ 0.60935985,  0.79902993],
       [ 0.58011688,  0.76262681],
       [ 0.55158962,  0.74071726],
       [ 0.54425652,  0.79401891],
       [ 0.60790559,  0.76123551]])

In [33]:
all_X[-1][-30:].flatten()

array([ 0.60935985,  0.58011688,  0.55158962,  0.54425652,  0.60790559,
        0.61305799,  0.59323257,  0.64445579,  0.66066766,  0.63381986,
        0.64185573,  0.67199441,  0.73598931,  0.73157587,  0.69488388,
        0.79248418,  0.87443477,  0.83571034,  0.85276064,  0.87911829,
        0.87481689,  0.82992736,  0.83403793,  0.83011021,  0.79484352,
        0.79902993,  0.76262681,  0.74071726,  0.79401891,  0.76123551])

In [8]:
# import numpy as np
# from sklearn.base import BaseEstimator
# from sklearn.utils.validation import check_X_y, check_array, check_is_fitted


# class DtrajReshape(BaseEstimator):
#     """ A template estimator to be used as a reference implementation .
#     Parameters
#     ----------
#     demo_param : str, optional
#         A parameter used for demonstation of how to pass and store paramters.
#     """
#     def __init__(self):
#         pass

#     def fit(self, X, y=None):
#         """A reference implementation of a fitting function
#         Parameters
#         ----------
#         X : array-like or sparse matrix of shape = [n_samples, n_features]
#             The training input samples.
#         y : array-like, shape = [n_samples] or [n_samples, n_outputs]
#             The target values (class labels in classification, real numbers in
#             regression).
#         Returns
#         -------
#         self : object
#             Returns self.
#         """
#         # X, y = check_X_y(X, y)


#         # Return the estimator
#         return self

#     def transform(self, X):
#         """ A reference implementation of a predicting function.
#         Parameters
#         ----------
#         X : array-like of shape = [n_samples, n_features]
#             The input samples.
#         Returns
#         -------
#         y : array of shape = [n_samples]
#             Returns :math:`x^2` where :math:`x` is the first column of `X`.
#         """
#         X = check_array(X)
#         X_list = [X[i,:].T[:, np.newaxis] for i in range(X.shape[0])]
#         return X_list

In [9]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV


# traj_paths = glob('data/000.5pc/*.npy')
# X = [np.load(traj_path) for traj_path in traj_paths]

xmin, xmax = -1.2, 1.2
tau = 25

model_reshape = Pipeline([('reshape',  DtrajReshape()), 
                  ('cluster',NDGrid(min=xmin, max=xmax, n_bins_per_feature=200)),
                  ('msm', MaximumLikelihoodMSM(lag=1, score_method='vamp1', score_k=3))])

model = Pipeline([('cluster',NDGrid(min=xmin, max=xmax, n_bins_per_feature=200)),
                  ('msm', MaximumLikelihoodMSM(lag=tau, score_method='vamp1', score_k=3))])

param_grid={'cluster__n_bins_per_feature': [100,150,200]}
search_reshape = GridSearchCV(model_reshape, param_grid=param_grid, cv=3)
search = GridSearchCV(model, param_grid=param_grid, cv=3)

In [10]:
model.fit([X])

Pipeline(memory=None,
     steps=[('cluster', NDGrid(max=1.2, min=-1.2, n_bins_per_feature=200)), ('msm', MaximumLikelihoodMSM(connectivity='largest', count_mode='sliding',
           dt_traj='1 step', lag=25, maxerr=1e-08, maxiter=1000000,
           mincount_connectivity='1/n', reversible=True, score_k=3,
           score_method='vamp1', sparse=False, statdist_constraint=None))])

In [11]:
model_reshape.fit(X_split)

Pipeline(memory=None,
     steps=[('reshape', DtrajReshape()), ('cluster', NDGrid(max=1.2, min=-1.2, n_bins_per_feature=200)), ('msm', MaximumLikelihoodMSM(connectivity='largest', count_mode='sliding',
           dt_traj='1 step', lag=1, maxerr=1e-08, maxiter=1000000,
           mincount_connectivity='1/n', reversible=True, score_k=3,
           score_method='vamp1', sparse=False, statdist_constraint=None))])

In [12]:
search_reshape.fit(X_split)

GridSearchCV(cv=3, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('reshape', DtrajReshape()), ('cluster', NDGrid(max=1.2, min=-1.2, n_bins_per_feature=200)), ('msm', MaximumLikelihoodMSM(connectivity='largest', count_mode='sliding',
           dt_traj='1 step', lag=1, maxerr=1e-08, maxiter=1000000,
           mincount_connectivity='1/n', reversible=True, score_k=3,
           score_method='vamp1', sparse=False, statdist_constraint=None))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'cluster__n_bins_per_feature': [100, 150, 200]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [13]:
import pandas as pd
df = pd.DataFrame(search_reshape.cv_results_)

/Users/robert_arbon/anaconda/envs/ml4dyn/lib/python3.5/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('mean_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/Users/robert_arbon/anaconda/envs/ml4dyn/lib/python3.5/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/Users/robert_arbon/anaconda/envs/ml4dyn/lib/python3.5/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_

In [14]:
df.head()

,mean_fit_time,mean_score_time,mean_test_score,mean_train_score,param_cluster__n_bins_per_feature,params,rank_test_score,split0_test_score,split0_train_score,split1_test_score,split1_train_score,split2_test_score,split2_train_score,std_fit_time,std_score_time,std_test_score,std_train_score
0,0.044501,0.024255,1.325131,1.999357,100,{'cluster__n_bins_per_feature': 100},2,1.256461,2.062304,1.373406,2.111139,1.346045,1.824627,0.008926,0.004775,0.050001,0.125150
1,0.040330,0.040160,1.177056,2.191376,150,{'cluster__n_bins_per_feature': 150},3,1.093941,2.194015,1.151039,2.313545,1.286820,2.066568,0.008418,0.026637,0.080903,0.100845
2,0.050140,0.034273,1.361903,2.318269,200,{'cluster__n_bins_per_feature': 200},1,1.185844,2.354765,1.331244,2.423590,1.569954,2.176451,0.010812,0.001630,0.158395,0.104142


In [15]:
model.named_steps['msm'].timescales(k=4)

array([ 55.74457142,  53.62113347,  43.03708525,  42.20639984])

In [16]:
model_reshape.named_steps['msm'].timescales(k=4)*25

array([ 55.74457142,  53.62113347,  43.03708525,  42.20639984])